# Optimización Cuántica para la Trayectoria de Ascenso de Aeronaves

Este notebook presenta una implementación de optimización de la trayectoria de ascenso utilizando computación cuántica (simulada) y clásica. El objetivo es minimizar una función de costo que representa el consumo de combustible y el tiempo de vuelo, considerando restricciones operacionales.

In [ ]:
import numpy as np
from scipy.optimize import minimize
import sympy as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import dimod
from pyqubo import Binary, Array
from itertools import product

def Fun(x): #Funcion principal de costo
    # Código detallado de la función de costo (omitido para brevedad pero mostrada en el repositorio)
    # Debe incluir las definiciones de Zp, F_N_MCL, ρ, M, CAS, etc.,
    # así como la lógica para calcular la función de costo.
    return φ(np.concatenate(([v_0], x1)), np.concatenate(([γ_0], x2*π/180)))

def FF(x):
    a = -Fun(x)
    if a > 0:
        return a
    else:
        return 58626 # Valor mas alto en la funcion

print(FF([188.5, 1.7]))

#Se crea una funcion que trabaje igualmente con arrays
def fun(x, y):
    if np.isscalar(x) and np.isscalar(y):
        return FF([x, y])
    else:
        coords = np.stack((x, y), axis=-1)
        vectorized_funn = np.vectorize(FF, signature='(n)->()')
        return vectorized_funn(coords)

x = np.array([[188, 1.7], [3, 4]])
y = np.array([[5, 6], [7, 8]])

resultado = fun(x, y)
print(resultado)

## Exploración de Puntos Funcionales
El siguiente código busca puntos funcionales dentro de un intervalo dado, utilizando una estrategia de búsqueda aleatoria con perturbaciones gaussianas y un filtro basado en la media y desviación estándar de los valores de la función.

In [ ]:
def buscar_puntos(f, punto_inicial, intervalo, n, nb, c):
    puntos_encontrados = []
    punto_actual = np.array(punto_inicial)
    intervalos = np.array(intervalo)
    varianza = c * (intervalos[:, 1] - intervalos[:, 0])

    for _ in range(nb):
        puntos = np.random.normal(loc=punto_actual, scale=varianza, size=(n, len(punto_inicial)))

        puntos = [p for p in puntos if all(intervalos[i, 0] <= p[i] <= intervalos[i, 1] for i in range(len(p)))]

        puntos_evaluados = [(p, f(*p)) for p in puntos]

        # Calcular la media de los valores de f
        if len(puntos_evaluados) > 0:  # Evitar la división por cero si no hay puntos
            media_f = np.mean([valor_f for _, valor_f in puntos_evaluados])

            # Filtrar puntos con f > media + 1.5*desviación estándar
            puntos_evaluados = [(p, valor_f) for p, valor_f in puntos_evaluados if valor_f <= media_f + 1.2 * np.std([valor_f for _, valor_f in puntos_evaluados])]

        puntos_evaluados.sort(key=lambda x: x[1])

        # Solo añadir los puntos que pasaron el filtro
        puntos_encontrados.extend([punto for punto, _ in puntos_evaluados])

        if len(puntos_encontrados) > 0:  # Evitar error si no hay puntos después del filtro.
            punto_actual = np.mean(puntos_encontrados, axis=0)


    return puntos_encontrados

## Visualización de la Búsqueda
Se grafica la distribución de los puntos encontrados en el espacio de búsqueda para visualizar la convergencia del algoritmo.

In [ ]:
puntos = buscar_puntos(fun, [188, 1.7], [[110,250],[0,20]],100, 10, 0.1)
print(puntos)

In [ ]:
x = [punto[0] for punto in puntos]
y = [punto[1] for punto in puntos]

z = [0] * len(puntos)

# Crear la figura y el objeto de ejes 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Graficar los puntos
ax.scatter(x, y, z, c='r', marker='.')

# Etiquetas de los ejes
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Ajustar la vista (opcional, pero a menudo útil)
ax.view_init(elev=20, azim=-35) # Ajusta estos valores para rotar la vista

# Mostrar la gráfica
plt.show()

## Aproximación con Polinomios de Chebyshev
Se utiliza una aproximación de la función de costo con polinomios de Chebyshev para simplificar la optimización cuántica.

In [ ]:
def chebyshev_polynomial(n, x):
    """Generate symbolic Chebyshev polynomial of degree n"""
    if n == 0:
        return sp.Integer(1)
    elif n == 1:
        return x
    else:
        return 2 * x * chebyshev_polynomial(n - 1, x) - chebyshev_polynomial(n - 2, x)

def scale_to_chebyshev(x, interval):
    """Scale x from [a,b] to [-1,1]"""
    a, b = interval
    return 2 * (x - a) / (b - a) - 1

def polinomio_de_cheb(f, p, g):
    """
    Genera un polinomio de Chebyshev de grado g que aproxima la función f usando los puntos en p.

    Parámetros:
    - f: función original que se desea aproximar.
    - p: lista de puntos a utilizar para la aproximación.
    - g: grado del polinomio de Chebyshev.

    Retorna:
    - Un polinomio simbólico de Chebyshev en términos de x1, x2, ..., xn.
    """
    # Deduce el número de variables (dimensión) a partir del primer punto en p
    num_vars = len(p[0])
    variables = sp.symbols(f'x1:{num_vars + 1}')  # Crea x1, x2, ..., xn

    # Determina los intervalos de cada variable en los puntos p
    intervals = [(min([point[i] for point in p]), max([point[i] for point in p])) for i in range(num_vars)]

    # Escala los puntos en p al espacio [-1, 1] usando los intervalos determinados
    points_cheb = np.array([
        [scale_to_chebyshev(point[i], intervals[i]) for i in range(num_vars)]
        for point in p
    ])

    # Evalúa la función en cada punto en p
    f_values = np.array([f(*point) for point in p])

    # Construye la matriz de Vandermonde en el espacio de Chebyshev
    num_terms = sum(1 for i in range(g + 1) for j in range(g + 1) if i + j <= g)
    V = np.zeros((len(p), num_terms))
    idx = 0
    for indices in product(range(g + 1), repeat=num_vars):
        if sum(indices) <= g:
            term_val = np.ones(len(p))
            for var_index, degree in enumerate(indices):
                term_val *= np.polynomial.chebyshev.chebval(points_cheb[:, var_index], [0] * degree + [1])
            V[:, idx] = term_val
            idx += 1

    # Resuelve el sistema usando mínimos cuadrados
    coeffs = np.linalg.lstsq(V, f_values, rcond=None)[0]

    # Construye el polinomio simbólico en términos de las variables
    expr = sp.Integer(0)
    idx = 0
    for indices in product(range(g + 1), repeat=num_vars):
        if sum(indices) <= g:
            term = coeffs[idx]
            for var_index, degree in enumerate(indices):
                term *= chebyshev_polynomial(degree, scale_to_chebyshev(variables[var_index], intervals[var_index]))
            expr += term
            idx += 1

    # Simplifica la expresión simbólica
    expr = sp.simplify(expr)
    return expr


In [ ]:
polinomio_de_cheb(fun, puntos, 5)

In [ ]:
f_example = fun
grado = 5

# Obtener el polinomio de Chebyshev usando el código anterior
polinomio_chebyshev = polinomio_de_cheb(f_example, puntos, grado)

# Crear la función evaluable a partir del polinomio de Chebyshev
x1, x2 = sp.symbols('x1 x2')

polinomio_func = sp.lambdify((x1, x2), polinomio_chebyshev, 'numpy')

# Intervalo para la gráfica
interval_x = (0, 10)
interval_y = (0, 10)


interval_x = (150, 240)
interval_y = (0, 7)

#interval_z = (-2, 3)

resolucion = 50  # número de puntos en cada eje

# Generar malla para graficar
x_vals = np.linspace(*interval_x, resolucion)
y_vals = np.linspace(*interval_y, resolucion)
X, Y = np.meshgrid(x_vals, y_vals)

# Evaluar función original en la malla
Z_original = f_example(X, Y)
umbral = 58500  # Define tu umbral
# Reemplaza valores mayores al umbral con np.nan
Z_original = np.where(Z_original > umbral, np.nan, Z_original)


# Evaluar el polinomio de Chebyshev en la malla
Z_chebyshev = polinomio_func(X, Y)

# Graficar

fig = plt.figure(figsize=(18, 6))

# 1. Gráfica de la función original
ax1 = fig.add_subplot(131, projection='3d')
surf1 = ax1.plot_surface(X, Y, Z_original, cmap='viridis', alpha=0.8)
ax1.set_title('Función Original')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z')
###ax1.set_zlim(interval_z)

# 2. Gráfica del polinomio de Chebyshev
ax2 = fig.add_subplot(132, projection='3d')
surf2 = ax2.plot_surface(X, Y, Z_chebyshev, cmap='viridis', alpha=0.8)
ax2.set_title('Aproximación de Chebyshev')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('z')
ax2.set_ylim(0,7.2)
ax2.set_xlim(150,240)
#ax2.set_zlim((53850,53900))

# 3. Gráfica de la función original con los puntos de muestreo resaltados
ax3 = fig.add_subplot(133, projection='3d')
#surf3 = ax3.plot_surface(X, Y, Z_original, cmap='viridis', alpha=0.8)
ax3.scatter([p[0] for p in puntos], [p[1] for p in puntos],
            [f_example(p[0], p[1]) for p in puntos],
            color='red', marker='.',s=50, label='Puntos de Muestreo')
ax3.set_title('Función Original con Puntos de Muestreo')
ax3.set_xlabel('x')
###ax3.set_ylabel('y')
ax3.set_ylabel('y')
ax3.set_zlabel('z')
ax3.legend()

plt.tight_layout()
plt.show()


## Optimización Cuántica
Se implementa la optimización cuántica utilizando un simulador de recocido cuántico (D-Wave).



En el siguiente codigo se hace finalmente la optimizacion usando un computador cuantico (simulado), sobre la funcion para N=2 aproximada mediante los polinomios de chebychev udando 10 bits.

In [ ]:
def funcion_objetivo(x, y):
    return 3.00755171107551*x + 23.4759463762618*y + 57739.0525970849

def visualizar_funcion(x_valor, y_valor): #  Se pasan x_valor e y_valor para graficarlos.
    x = np.linspace(150, 240, 100)
    y = np.linspace(0, 7, 100)
    X, Y = np.meshgrid(x, y)
    Z = funcion_objetivo(X, Y)

    plt.figure(figsize=(10, 8))
    plt.contour(X, Y, Z, levels=20)
    plt.colorbar(label='f(x,y)')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Contorno de la función objetivo')
    plt.plot(x_valor, y_valor, 'r*', markersize=15, label='Solución encontrada') #  Graficar el punto encontrado.
    plt.legend()
    plt.show()

def crear_qubo(num_bits=4): # Parámetro para el número de bits
    x_bits = Array.create('x', shape=num_bits, vartype='BINARY') #Usar array de PyQUBO facilita la suma.
    y_bits = Array.create('y', shape=num_bits, vartype='BINARY')

    x_min, x_max = 150, 240
    y_min, y_max = 0, 7

    x_scale = (x_max - x_min) / (2**num_bits - 1)
    y_scale = (y_max - y_min) / (2**num_bits - 1)

    x = x_min + x_scale * sum(2**i * x_bits[i] for i in range(num_bits))
    y = y_min + y_scale * sum(2**i * y_bits[i] for i in range(num_bits))

    H = 3.00755171107551*x + 23.4759463762618*y + 57739.0525970849

    model = H.compile()
    bqm = model.to_bqm()

    return bqm

def resolver_y_mostrar_resultados(num_bits=4, num_reads=1000):  #  Parámetros para bits y lecturas
    bqm = crear_qubo(num_bits)

    sampler = dimod.SimulatedAnnealingSampler()
    sampleset = sampler.sample(bqm, num_reads=num_reads) # Usar num_reads

    mejor_muestra = sampleset.first.sample

    x_min, x_max = 150, 240
    y_min, y_max = 0, 7

    x_scale = (x_max - x_min) / (2**num_bits - 1)
    y_scale = (y_max - y_min) / (2**num_bits - 1)

    x_valor = x_min + x_scale * sum(2**i * mejor_muestra[f'x[{i}]'] for i in range(num_bits)) #  Acceso al array con [i].
    y_valor = y_min + y_scale * sum(2**i * mejor_muestra[f'y[{i}]'] for i in range(num_bits)) #  Acceso al array con [i].


    print(f"\nMejor solución encontrada:")
    print(f"x = {x_valor:.4f}")
    print(f"y = {y_valor:.4f}")
    print(f"f(x,y) = {funcion_objetivo(x_valor, y_valor):.4f}")

    visualizar_funcion(x_valor, y_valor) #  Se pasa x_valor e y_valor.


if __name__ == "__main__":
    print("Optimizando función cuadrática usando D-Wave simulado...")

    resolver_y_mostrar_resultados(num_bits=10, num_reads=10)  #  Mayor número de bits y lecturas.

## Optimización Clásica
Se implementa la optimización clásica como punto de comparación, utilizando el método Nelder-Mead.

In [ ]:
def optimizar_no_suave(fun, x0, y0, bounds_x, bounds_y, method='Nelder-Mead', reinicios=5):
    """Optimiza una función no suave fun(x, y)."""

    def fun_wrapper(params):
        x, y = params
        return fun(x, y)

    punto_inicial = np.array([x0, y0])
    bounds = [bounds_x, bounds_y]

    mejor_resultado = None
    mejor_valor = np.inf

    for _ in range(reinicios):  # Reinicia la optimización varias veces
        result = minimize(fun_wrapper, punto_inicial, bounds=bounds, method=method)

        if result.success and result.fun < mejor_valor:  # Guarda el mejor resultado
            mejor_resultado = result
            mejor_valor = result.fun

        # Genera un nuevo punto inicial aleatorio dentro de los límites
        punto_inicial = np.array([
            np.random.uniform(bounds_x[0], bounds_x[1]),
            np.random.uniform(bounds_y[0], bounds_y[1])
        ])

    if mejor_resultado:
        x_optimo, y_optimo = mejor_resultado.x
        print("x óptimo:", x_optimo)
        print("y óptimo:", y_optimo)
        print("Valor óptimo:", mejor_resultado.fun)
        return x_optimo, y_optimo
    else:
        print("La optimización no convergió en ninguna de las iteraciones.")
        return None




x_inicial = 180
y_inicial = 1.7
limites_x = (150, 240)
limites_y = (0, 7)

x_opt, y_opt = optimizar_no_suave(fun, x_inicial, y_inicial, limites_x, limites_y)